# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1dd0382f40>
├── 'a' --> tensor([[ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911]])
└── 'x' --> <FastTreeValue 0x7f1dd0382e20>
    └── 'c' --> tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                        [-0.5280, -1.1438,  0.5143,  0.9539],
                        [-1.5864, -2.5816, -1.0950,  2.2560]])

In [4]:
t.a

tensor([[ 1.1529,  0.5996,  0.0835],
        [-0.7704,  0.1742,  0.5911]])

In [5]:
%timeit t.a

58.5 ns ± 0.0613 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1dd0382f40>
├── 'a' --> tensor([[-0.7513,  0.1209,  1.0225],
│                   [ 0.8742,  0.5297, -0.3420]])
└── 'x' --> <FastTreeValue 0x7f1dd0382e20>
    └── 'c' --> tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                        [-0.5280, -1.1438,  0.5143,  0.9539],
                        [-1.5864, -2.5816, -1.0950,  2.2560]])

In [7]:
%timeit t.a = new_value

59.1 ns ± 0.0531 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1529,  0.5996,  0.0835],
               [-0.7704,  0.1742,  0.5911]]),
    x: Batch(
           c: tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1529,  0.5996,  0.0835],
        [-0.7704,  0.1742,  0.5911]])

In [11]:
%timeit b.a

53.9 ns ± 0.0389 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8553, -1.0374, -2.0697],
               [-1.1464,  0.4795, -0.5136]]),
    x: Batch(
           c: tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.275 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.213 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 585 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1d1b623f70>
├── 'a' --> tensor([[[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]],
│           
│                   [[ 1.1529,  0.5996,  0.0835],
│                    [-0.7704,  0.1742,  0.5911]]])
└── 'x' --> <FastTreeValue 0x7f1d1b623cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 74.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1d1b623a90>
├── 'a' --> tensor([[ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911],
│                   [ 1.1529,  0.5996,  0.0835],
│                   [-0.7704,  0.1742,  0.5911]])
└── 'x' --> <FastTreeValue 0x7f1d1b623fa0>
    └── 'c' --> tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                        [-0.5280, -1.1438,  0.5143,  0.9539],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 39.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.4 µs ± 98.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6324,  0.5978, -0.0755, -0.0804],
                       [-0.5280, -1.1438,  0.5143,  0.9539],
                       [-1.5864, -2.5816, -1.0950,  2.2560]],
              
                      [[ 0.6324,  0.5978, -0.0755, -0.0804],
                       [-0.5280, -1.1438,  0.5143,  0.9539],
                       [-1.5864, -2.5816, -1.0950,  2.2560]],
              
                      [[ 0.6324,  0.5978, -0.0755, -0.0804],
                       [-0.5280, -1.1438,  0.5143,  0.9539],
                       [-1.5864, -2.5816, -1.0950,  2.2560]],
              
                      [[ 0.6324,  0.5978, -0.0755, -0.0804],
                       [-0.5280, -1.1438,  0.5143,  0.9539],
                       [-1.5864, -2.5816, -1.0950,  2.2560]],
              
                      [[ 0.6324,  0.5978, -0.0755, -0.0804],
                       [-0.5280, -1.1438,  0.5143,  0.9539],
                       [-1.5864, -2.5816, -1.0950,  2.2560]],

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560],
                      [ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560],
                      [ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560],
                      [ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560],
                      [ 0.6324,  0.5978, -0.0755, -0.0804],
                      [-0.5280, -1.1438,  0.5143,  0.9539],
                      [-1.5864, -2.5816, -1.0950,  2.2560],
                      [ 0.6324,  0.5978, -0.0755, -0.0804],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
